In [10]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as ln
import colour as cl

In [11]:
T_srgb_to_cie=np.array([[0.4124,0.3576,0.1805],
                    [0.2126,0.7152,0.0722],
                    [0.0193,0.1192,0.9505]])

T_cieXYZ_to_srgb = ln.inv(T_srgb_to_cie)

T_cieXYZ_to_p3d65 = np.array([[2.493497, -0.931384, -0.402711],
                              [-0.829489, 1.762664,  0.023625],
                              [0.035846, -0.076172,  0.956885]])
                              
T_p3d65_to_srgb = ln.inv(T_cieXYZ_to_p3d65)

cmf_XYZ = np.genfromtxt('lin2012xyz10e_1_7sf.csv', delimiter=',')

cmf_p3d65 = np.dot(T_cieXYZ_to_p3d65,cmf_XYZ[:,1:].T)
cmf_p3d65 = np.vstack((cmf_XYZ[:,0],cmf_p3d65)).T

cmf_srgb = np.dot(T_cieXYZ_to_srgb,cmf_XYZ[:,1:].T)
cmf_srgb = np.vstack((cmf_XYZ[:,0],cmf_srgb)).T

In [12]:
def inner(spectrum, cmf):
    """
    spectrum shuold be (m,n) array, row is spectrum

    cmf should be (n,3) array

    return (m,3)
    """
    R = np.inner(spectrum, cmf[:,0])
    G = np.inner(spectrum, cmf[:,1])
    B = np.inner(spectrum, cmf[:,2])
    rgb = np.vstack((R,G,B)).T
    rgb = rgb/np.max(rgb)
    return rgb

In [13]:
APD = np.genfromtxt('APD_CMF.csv',delimiter=',') # from 360 to 730
APD = APD[8:] # reduced to 367 to 730
xrite_spectrum = np.genfromtxt('xrite_cherker_spectrum.csv',delimiter=' ') # 24colors 
xrite_spectrum_1nm = np.repeat(xrite_spectrum, repeats=10, axis=1)[:,390-376:] # interplot to 1nm, from 376 to 735, reduced to 390 to 735 for p3-D65 CMF
xrite_spectrum_2nm = np.repeat(xrite_spectrum, repeats=5, axis=1)[:,:(730-376)//2+1] # interplot to 2nm, from 376 to 734, reduced to 376 to 730, for APD CMF
cmf_p3d65_390_735 = cmf_p3d65[:735-390+1]
cmf_XYZ_390_735 = cmf_XYZ[:735-390+1]

In [14]:
xr_p3 = inner(xrite_spectrum_1nm,cmf_p3d65_390_735[:,1:])
xr_apd = inner(xrite_spectrum_2nm,APD[:,1:])

In [15]:
xrite_spectrum_2nm.shape

(24, 178)

In [16]:
ccm_p3_to_apd = ln.lstsq(xr_p3,xr_apd)[0].T

/var/folders/cv/3w4fv3096tq_v6gkm2b1x3vm0000gn/T/ipykernel_19750/1865324280.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  ccm_p3_to_apd = ln.lstsq(xr_p3,xr_apd)[0].T


In [17]:
ccm_p3_to_apd

array([[ 1.43125479, -0.94726934,  0.59132499],
       [ 0.00718712,  0.58665976, -0.06972452],
       [-0.01965487,  0.06406901,  0.74042105]])